In [1]:
import pandas as pd
from pandas import Series, DataFrame
import country_converter as coco
import string
import os
import numpy as np

In [2]:
path = '/Users/ariana/desktop/historical_tech'
os.chdir(path)

In [3]:
# https://www.usgs.gov/centers/national-minerals-information-center/historical-global-statistics-mineral-and-material
# Data Series 896

In [4]:
def read_usgs_896(tech_name, file_name, sheet_name=0):
    df = pd.read_excel(file_name, sheet_name=sheet_name, header=3,
                       na_values=['XX','--', 'W', '(3/)', '(4/)','-- '])
    drop_idx = []
    for idx in df.index:
        country = df[df.columns[0]].iloc[idx]
        drop_list = ['Limonite', 'Other', 'Puerto Rico', 'United States: Puerto Rico', 'Leeward and Windward Islands ']
        if country in drop_list:
            drop_idx.append(idx)
    df.drop(index=drop_idx, inplace=True)
    df.reset_index(drop=True, inplace=True)
    for idx in df.index:
        country = df[df.columns[0]].iloc[idx]
        if country == 'World':
            world_idx = idx
    df = df[:world_idx]
    countries = []
    for country in df[df.columns[0]]:
        country = country.strip(string.digits)
        country = country.strip()
        countries.append(country)
    df['Country Name'] = countries
    df.drop(columns=df.columns[0], inplace=True)
    iso2 = []
    spatial_scale = []
    for country in df['Country Name']:
        if country == 'Serbia and Montenegro':
            iso2.append('CS')
            spatial_scale.append('National')
        elif country == 'Czechoslovakia':
            iso2.append('CSK')
            spatial_scale.append('National')
        elif country == 'U.S.S.R.':
            iso2.append('SU')
            spatial_scale.append('National')
        elif country == 'Yugoslavia':
            iso2.append('YU')
            spatial_scale.append('National')
        elif country == 'Netherlands Antilles':
            iso2.append('AN')
            spatial_scale.append('National')
        elif country == 'World':
            iso2.append('World')
            spatial_scale.append('Global')
        else:
            iso2.append(coco.convert(names=country, to='iso2'))
            spatial_scale.append('National')
    df['Country Code'] = iso2
    df['Spatial Scale'] = spatial_scale
    df['Data Source'] = 'USGS'
    df['Unit'] = 'metric tons'
    df['Metric'] = 'Annual production'
    df['Technology Name'] = tech_name
    df['ID'] = df['Technology Name'] + '_' + df['Metric'] + '_' + df['Country Code']
    df.set_index('ID', inplace=True)
    omit = []
    for col in df.columns:
        if len(str(col)) > 4 or col == 'Unit':
            omit.append(col)
    empty_rows = df.drop(columns=omit)
    empty_rows.dropna(how='all', inplace=True)
    na_idx = []
    for country in df.index:
        if country not in empty_rows.index:
            na_idx.append(country)
    df.drop(na_idx, inplace=True)
    file_name = 'cleaned data/ds896_' + tech_name + '.csv'
    df.to_csv(file_name)
    return df


In [5]:
# Aluminum and Bauxite
aluminum = read_usgs_896('Primary Aluminum Production', 'raw data/usgs/ds896-aluminum.xlsx', 'Aluminum, primary production')
bauxite = read_usgs_896('Primary Bauxite Production', 'raw data/usgs/ds896-aluminum.xlsx', 'Bauxite')


/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [6]:
# Cadmium
# Omit East/West Germany since there's only one year of data for each
cadmium = read_usgs_896('Cadmium Refining', 'raw data/usgs/ds896-cadmium.xlsx')
cadmium = cadmium[(cadmium['Country Name']!='Germany: Eastern states') & (cadmium['Country Name']!='Germany: Western states')]
cadmium.to_csv('cleaned data/ds896_Cadmium Refining.csv')


/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [7]:
# Copper mining and refining
copper_mining = read_usgs_896('Copper|Mining', 'raw data/usgs/ds896-copper.xlsx', 'Mine')
copper_mining = copper_mining[copper_mining['    Mine production']=='Mine: Total']
copper_mining.drop(columns='    Mine production', inplace=True)
copper_mining.to_csv('cleaned data/ds896_Copper|Mining.csv')

copper_refining = read_usgs_896('Copper|Refining', 'raw data/usgs/ds896-copper.xlsx', 'Refinery')
copper_refining = copper_refining[copper_refining['Refinery production']=='Refinery: Total']
copper_refining.drop(columns='Refinery production', inplace=True)
copper_refining.to_csv('cleaned data/ds896_Copper|Refining.csv')

In [8]:
# Iron ore and raw steel
# Omitting West Germany because there's only one data point
iron = read_usgs_896('Iron Ore', 'raw data/usgs/ds896-iron-steel.xlsx', 'Iron ore, gross weight')
iron = iron[(iron['Country Name']!='Germany: Western states')]
iron.to_csv('cleaned data/ds896_Iron ore.csv')

steel = read_usgs_896('Raw Steel Production', 'raw data/usgs/ds896-iron-steel.xlsx', 'Raw steel')
steel = steel[(steel['Country Name']!='Germany: Eastern states') & (steel['Country Name']!='Germany: Western states')]
steel.to_csv('cleaned data/ds896_Raw steel production.csv')

/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [9]:
# Lead
lead = read_usgs_896('Lead', 'raw data/usgs/ds896-lead.xlsx', 'Mine')
lead = lead[lead['Country Name']!='Germany: Western states']
lead.to_csv('cleaned data/ds896_Lead.csv')

/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [10]:
# Salt
salt = pd.read_excel('raw data/usgs/ds896-salt.xlsx', sheet_name=0, header=3,
                     na_values=['XX','--', 'W', '(3/)', '(4/)','-- '])
drop_idx = []
for idx in salt.index:
    country = salt[salt.columns[0]].iloc[idx]
    drop_list = ['Limonite', 'Other', 'Puerto Rico', 'United States: Puerto Rico', 'Leeward and Windward Islands ']
    if country in drop_list:
        drop_idx.append(idx)
salt.drop(index=drop_idx, inplace=True)
salt.reset_index(drop=True, inplace=True)
for idx in salt.index:
    country = salt[salt.columns[0]].iloc[idx]
    if country == 'World':
        world_idx = idx
salt = salt[:world_idx]
countries = []
for country in salt[salt.columns[0]]:
    country = country.strip(string.digits)
    country = country.strip()
    countries.append(country)
salt['Country Name'] = countries
salt = salt[(salt['Country Name']!='Germany: Eastern states') & (salt['Country Name']!='Germany: Western states')]
salt.drop(columns=[salt.columns[0], 'Salt type'], inplace=True)
unique_countries = sorted(list(set(salt['Country Name'])))
salt_type_sum = []
for country in unique_countries:
    dup_df = salt[salt['Country Name']==country].sum()
    dup_df = dup_df[:-1]
    val_list = []
    for x in dup_df:
        val_list.append(x)
    val_list.append(country)
    salt_type_sum.append(val_list)
salt = pd.DataFrame(salt_type_sum, columns=salt.columns)
iso2 = []
spatial_scale = []
for country in salt['Country Name']:
    if country == 'Serbia and Montenegro':
        iso2.append('CS')
        spatial_scale.append('National')
    elif country == 'Czechoslovakia':
        iso2.append('CSK')
        spatial_scale.append('National')
    elif country == 'U.S.S.R.':
        iso2.append('SU')
        spatial_scale.append('National')
    elif country == 'Yugoslavia':
        iso2.append('YU')
        spatial_scale.append('National')
    elif country == 'World':
        iso2.append('World')
        spatial_scale.append('Global')
    elif country == 'Netherlands Antilles':
        iso2.append('AN')
        spatial_scale.append('National')
    else:
        iso2.append(coco.convert(names=country, to='iso2'))
        spatial_scale.append('National')
salt['Country Code'] = iso2
salt['Spatial Scale'] = spatial_scale
salt['Data Source'] = 'USGS'
salt['Unit'] = 'metric tons'
salt['Metric'] = 'Annual production'
salt['Technology Name'] = 'Salt Production'
salt['ID'] = salt['Technology Name'] + '_' + salt['Metric'] + '_' + salt['Country Code']
salt.set_index('ID', inplace=True)
salt.replace(0, np.nan, inplace=True)
salt.to_csv('cleaned data/ds896_Salt production.csv')

In [11]:
# https://www.usgs.gov/centers/national-minerals-information-center/historical-statistics-mineral-and-material-commodities
# Data Series 140


In [12]:
def read_usgs_140(tech_name, file_name, sheet=0, header=4,
                  col_names=['Year','Production','Unit value (98$/t)','World production']):
    df = pd.read_excel(file_name, sheet_name=sheet, header=header, usecols=col_names, 
                       na_values=['XX','--', 'W', '(3/)', '(4/)','-- '])
    end_idx = None
    for idx in df.index:
        year = df['Year'].iloc[idx]
        if type(year) != int:
            end_idx = idx
            break
    df = df[:end_idx]
    df.set_index('Year', drop=True, inplace=True)
    df = df.transpose()
    df['Technology Name'] = tech_name
    df['Data Source'] = 'USGS'
    df['Country Name'] = ['United States', 'World', 'World']
    df['Country Code'] = ['US', 'World', 'World']
    df['Spatial Scale'] = ['National', 'Global', 'Global']
    df['Unit'] = ['metric tons', '1998 USD/metric ton', 'metric tons']
    df['Metric'] = ['Annual production', 'Price', 'Annual production']
    df['ID'] = df['Technology Name'] + '_' + df['Metric'] + '_' + df['Country Code']
    df.set_index('ID', drop=True, inplace=True)
    df.columns.name = None
    file_name = 'cleaned data/ds140_' + tech_name + '.csv'
    df.to_csv(file_name)
    return df


In [13]:
# raw steel
steel_ds140 = read_usgs_140('Raw Steel Production', 'raw data/usgs/ds140-iron-steel-2019.xlsx', header=5,
                            sheet='Steel',
                            col_names=['Year','Raw steel production','Unit value (98$/t)','World production'])

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [14]:
# cadmium
cadmium_ds140 = read_usgs_140('Cadmium Refining', 'raw data/usgs/ds140-cadmium-2019.xlsx',
                             col_names=['Year','Production','Unit value (98 $/t)','World production'])

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [15]:
# iron ore
iron_ds140 = read_usgs_140('Iron Ore', 'raw data/usgs/ds140-iron-ore-2019.xlsx',
                          col_names=['Year','Production','Unit value (98$/t) ','World production'])

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [16]:
# lead
lead_ds140 = read_usgs_140('Lead', 'raw data/usgs/ds140-lead-2018.xlsx',
                          col_names=['Year','Primary production','Unit value (98$/t)','World production'])

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [17]:
# primary aluminum
aluminum_ds140 = read_usgs_140('Primary Aluminum Production', 'raw data/usgs/ds140-aluminum-2019.xlsx',
                          col_names=['Year','Primary production','Unit value (98$/t)','World production'])

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [18]:
# graphite
graphite = read_usgs_140('Graphite', 'raw data/usgs/ds140-graphite-2019.xlsx')

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [19]:
# cobalt
cobalt = read_usgs_140('Cobalt', 'raw data/usgs/ds140-cobalt-2019.xlsx',
                    col_names=['Year','Primary production','Unit value (98$/t)','World mine production'])


/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [20]:
# lithium
lithium = read_usgs_140('Lithium Mine Production', 'raw data/usgs/ds140-lithium-2019.xlsx',
                    col_names=['Year','Production','Unit value (98$/t)','World production (gross weight)'])

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [21]:
# rare earth
rare_earth = read_usgs_140('Rare Earth Mine Production', 'raw data/usgs/ds140-rare-earths-2019.xlsx')

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [22]:
# silver
silver = read_usgs_140('Silver', 'raw data/usgs/ds140-silver-2018.xlsx',
                    col_names=['Year','Mine production','Unit value (98$/t)','World production'])

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [23]:
# tin
tin = read_usgs_140('Tin', 'raw data/usgs/ds140-tin-2019.xlsx',
                    col_names=['Year','Primary production','Unit value (98$/t)','World production'])

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [24]:
# Cement
cement = read_usgs_140('Cement', 'raw data/usgs/ds140-cement-2019.xlsx')

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [25]:
# Primary copper
copper = read_usgs_140('Primary Copper', 'raw data/usgs/ds140-copper-2018.xlsx', 
             col_names=['Year','Primary production','Unit value (98$/t)','World production'])

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [26]:
# Gold
gold = read_usgs_140('Gold', 'raw data/usgs/ds140-gold-2018.xlsx', 
             col_names=['Year','Primary production','Unit value (98$/t)','World production'])

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [27]:
# Nickel
nickel = read_usgs_140('Nickel Production', 'raw data/usgs/ds140-nickel-2018_1.xlsx', 
             col_names=['Year','Primary production','Unit value (98$/t)','World production'])

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [28]:
# Sand and gravel (construction)
sand_gravel_construction = read_usgs_140('Sand and Gravel|Construction', 
                                         'raw data/usgs/ds140-sand-gravel-construction-2019.xlsx')

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [29]:
# Sand and gravel (industrial)
sand_gravel_industrial = read_usgs_140('Sand and Gravel|Industrial', 
                                         'raw data/usgs/ds140-sand-industrial-2019.xlsx')

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [30]:
# zinc
zinc = read_usgs_140('Zinc', 'raw data/usgs/ds140-zinc-2019.xlsx',
                    col_names=['Year',' Primary production','Unit value (98$/t)','World production'])

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [31]:
# salt
salt_ds140 = read_usgs_140('Salt Production', 'raw data/usgs/ds140-2017-salt.xlsx',
                    col_names=['Year','Production','Unit value  (98$/t)','World production'])

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [32]:
# bauxite
bauxite_ds140 = read_usgs_140('Primary Bauxite Production', 'raw data/usgs/ds140-bauxite-alumina-2019.xlsx',
                          col_names=['Year','Production','Unit value (98$/t)','World production'])

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
